In [ ]:
import numpy as np
import pandas as pd

coord = (pd.read_csv("../data/airports.dat", index_col=['AIRPORT'], 
                     usecols=['AIRPORT','LATITUDE','LONGITUDE'])
         .groupby(level=0)
         .first()
         .dropna()
         .sample(n=500, random_state=42)
         .sort_index())

coord.head()

In [ ]:
idx = pd.MultiIndex.from_product([coord.index, coord.index], 
                                             names=['origin','dest'])

pairs = pd.concat([coord.add_suffix('_SRC').reindex(idx, level='origin'),
                   coord.add_suffix('_DST').reindex(idx, level='dest')],
                  axis=1)

pairs.head()

In [ ]:
idx = idx[idx.get_level_values(0) <= idx.get_level_values(1)]

idx

In [ ]:
import math

def gcd_py(lat_src, lng_src, lat_dst, lng_dst):
    earth_radius_km = 6373
    degs_to_rads = math.pi/180.0
    precision = 8 #dp
    
    theta_1 = (90-lat_src) * degs_to_rads
    theta_2 = (90-lat_dst) * degs_to_rads
    
    omega_1 = lng_src * degs_to_rads
    omega_2 = lng_dst * degs_to_rads
    
    cos = (math.sin(theta_1) * math.sin(theta_2) * 
           math.cos(omega_1 - omega_2) + 
           math.cos(theta_1) * math.cos(theta_2))
    
    cos = round(cos, precision)
    arc = math.acos(cos)
    
    return arc * earth_radius_km



In [ ]:
%%time
r = pairs.apply(lambda x: gcd_py(x['LATITUDE_SRC'], x['LONGITUDE_SRC'], 
                                 x['LATITUDE_DST'], x['LATITUDE_DST']), axis=1)

In [ ]:
r.head()

In [ ]:
def gcd_vec(lat_src, lng_src, lat_dst, lng_dst):
    earth_radius_km = 6373
    
    theta_1 = np.deg2rad(90 - lat_src)
    theta_2 = np.deg2rad(90 - lat_dst)
    
    omega_1 = np.deg2rad(lng_src)
    omega_2 = np.deg2rad(lng_dst)
    
    cos = (np.sin(theta_1) * np.sin(theta_2) * np.cos(omega_1 - omega_2) +
          np.cos(theta_1) * np.cos(theta_2))
    
    arc = np.arccos(cos)
    
    return arc * earth_radius_km



In [ ]:
%%time
r = pairs.apply(lambda x: gcd_vec(x['LATITUDE_SRC'], x['LONGITUDE_SRC'], 
                                  x['LATITUDE_DST'], x['LATITUDE_DST']), axis=1)

In [ ]:
r.head()

In [ ]:
%%time
pd.Series([gcd_py(*x) for x in pairs.itertuples(index=False)], 
                                            index=pairs.index)

In [ ]:
%%time
pd.Series([gcd_vec(*x) for x in pairs.itertuples(index=False)], index=pairs.index)

In [ ]:
%%time
r = gcd_vec(pairs['LATITUDE_SRC'], pairs['LONGITUDE_SRC'], 
            pairs['LATITUDE_DST'], pairs['LONGITUDE_DST'])

In [ ]:
r